In [1]:
__copyright__ = "Reiner Lemoine Institut gGmbH"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/eDisGo/blob/master/LICENSE"
__author__    = "gplssm, birgits"

# Welcome to the eDisGo example

eDisGo is a **python toolbox for the analysis of distribution networks** (low and medium voltage) that can be used to investigate economically viable **network expansion** scenarios, considering alternative flexibility options such as **storages or redispatch**. 

eDisGo is developed in the [open_eGo research project](https://openegoproject.wordpress.com/). It is based on [PyPSA](https://pypsa.org/), a toolbox for simulation and optimization of power networks, and closely related to the [ding0](https://dingo.readthedocs.io/en/dev/) project. ding0 stands for distribution network generator and is a tool to generate synthetic status quo medium and low voltage power distribution networks based on open (or at least accessible) data. It is currently the single data source for eDisGo providing synthetic grid data for whole Germany.

**! eDisGo is work in progress !** Please be aware that some of its features may still be buggy and not yet very sophisticated. We are happy for any bug reports, hints, etc. you may have for us.

### Learn more about eDisGo

* __[eDisGo Source Code](https://github.com/openego/eDisGo)__
* __[eDisGo Documentation](http://edisgo.readthedocs.io/en/dev/)__

### Table of Contents

* [The eDisGo API](#settings)
* [The eDisGo data container and grid data structure](#network)
* [Future generator capacities](#generator_scenario)
* [Grid reinforcement](#grid_reinforcement)
* [Evaluate results](#evaluation)
* [References](#references)

## About the example

This example shows the general usage of eDisGo. Grid expansion costs for an example distribution grid (see image below) are calculated assuming additional renewable energy generators as stated in the open_eGo 'NEP 2035' scenario (based on the scenario framework of the German grid development plan (Netzentwicklungsplan) for the year 2035) and conducting a worst-case analysis. Moreover, the eDisGo network data structure and how to access the results are introduced. At the end of the example grid expansion costs for a different scenario are calculated and compared to the grid expansion costs in the 'NEP 2035' scenario.

<img style="transform:rotate(90deg)" src="grid_1476.png" align="center">

**Let's get started!** First of all we have to import some packages.

#### Import packages

In [2]:
import os
import sys
import pandas as pd

from edisgo import EDisGo

import logging
logging.basicConfig(level=logging.ERROR)

/home/Birgit.Schachler/virtualenvs/open_ego_notebook/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
INFO:keyring.backend:Loading SecretService
INFO:keyring.backend:Loading kwallet
INFO:keyring.backend:Loading macOS
INFO:keyring.backend:Loading windows
INFO:keyring.backend:Loading Gnome
INFO:keyring.backend:Loading Google
INFO:keyring.backend:Loading Windows (alt)
INFO:keyring.backend:Loading file
INFO:keyring.backend:Loading keyczar
INFO:keyring.backend:Loading multi
INFO:keyring.backend:Loading pyfs


## The eDisGo API <a class="anchor" id="settings"></a>

The top-level API for setting up your scenario, invoking grid expansion and flexibility measures, etc. is provided by the **EDisGo class** (see [class documentation](http://edisgo.readthedocs.io/en/dev/api/edisgo.grid.html#edisgo.grid.network.EDisGo) for more information).

In this example we simply want to do a worst-case analysis of a ding0 grid. For this, we only have to provide a grid and set the 'worst_case_analysis' parameter. 

#### Specifying the ding0 grid

The ding0 grid is specified through the input parameter 'ding0_grid'. The following assumes you have a file of a ding0 grid named “ding0_grids__6.pkl” in current working directory.


In [3]:
ding0_grid = os.path.join(sys.path[0], "ding0_grids_239_DPv0.4.0.pkl")

#### Specifying worst-case analysis

As worst-case analysis you can either just analyse the feed-in or the load case or do a combined analysis. Choose between the following options:

* **’worst-case-feedin’** 
  
  Feed-in and demand for the worst-case scenario "feed-in case" are generated. Demand is set to 15% of maximum demand  for loads connected to the MV grid and 10% for loads connected to the LV grid. Feed-in of all generators is set to nominal power of the generator, except for PV systems where it is set to 85% of the nominal power.

  
* **’worst-case-load’**

  Feed-in and demand for the worst-case scenario "load case" are generated. Demand of all loads is set to maximum demand. Feed-in of all generators is set to zero.


* **’worst-case’**
  
  Feed-in and demand for the two worst-case scenarios "feed-in case" and "load case" are generated.

Instead of doing a worst-case analysis you can also provide your own timeseries for demand and feed-in and use those in the network analysis. EDisGo also offers methods to generate load and feed-in time series. Check out the [EDisGo class documentation](http://edisgo.readthedocs.io/en/dev/api/edisgo.grid.html#edisgo.grid.network.EDisGo) for more information.

In [4]:
worst_case_analysis = 'worst-case-feedin'

Now we are ready to initialize the edisgo API object.

In [5]:
edisgo = EDisGo(ding0_grid=ding0_grid,
                worst_case_analysis=worst_case_analysis)

## The eDisGo data container and grid data structure <a class="anchor" id="network"></a>

The last line, besides a couple of other things, initialized the [Network class](http://edisgo.readthedocs.io/en/dev/api/edisgo.grid.html#edisgo.grid.network.Network) which serves as an overall data container in eDisGo holding the grid data for the [MV grid](http://edisgo.readthedocs.io/en/dev/api/edisgo.grid.html#edisgo.grid.grids.MVGrid) and [LV grids](http://edisgo.readthedocs.io/en/dev/api/edisgo.grid.html#edisgo.grid.grids.LVGrid), [config data](http://edisgo.readthedocs.io/en/dev/api/edisgo.grid.html#edisgo.grid.network.Config), [results](http://edisgo.readthedocs.io/en/dev/api/edisgo.grid.html#edisgo.grid.network.Results), [timeseries](http://edisgo.readthedocs.io/en/dev/api/edisgo.grid.html#edisgo.grid.network.TimeSeries), etc. It is linked from multiple locations and provides hierarchical access to all data. Network itself can be accessed via the EDisGo API object as follows:

```python
edisgo.network
```

As mentioned *Network* holds the MV grid and LV grids. The grid topology is represented by separate undirected graphs for the MV grid and each of the LV grids. Each of these graphs is an eDisGo [Graph](http://edisgo.readthedocs.io/en/dev/_modules/edisgo/grid/grids.html#Graph), which is subclassed from networkx.Graph and extended by extra-functionality. Lines represent edges in the graph. Other equipment is represented by a node. Let's have a look into the graph.

First we take a look at all the **lines**.

In [6]:
# get a dictionary of all lines in the mv grid
edisgo.network.mv_grid.graph.edge

{GeneratorFluctuating_839361: {GeneratorFluctuating_878867: {'type': 'line',
   'line': Line_2390003},
  BranchTee_MVGrid_239_86: {'type': 'line', 'line': Line_2390004}},
 GeneratorFluctuating_839362: {BranchTee_MVGrid_239_87: {'type': 'line',
   'line': Line_2390005}},
 GeneratorFluctuating_839363: {BranchTee_MVGrid_239_88: {'type': 'line',
   'line': Line_2390006}},
 GeneratorFluctuating_839364: {LVStation_119612: {'type': 'line',
   'line': Line_2390007}},
 GeneratorFluctuating_878450: {BranchTee_MVGrid_239_89: {'type': 'line',
   'line': Line_2390008}},
 GeneratorFluctuating_878583: {BranchTee_MVGrid_239_84: {'type': 'line',
   'line': Line_2390001}},
 GeneratorFluctuating_878609: {MVStation_239: {'type': 'line',
   'line': Line_2390009}},
 GeneratorFluctuating_878611: {MVStation_239: {'type': 'line',
   'line': Line_2390010}},
 GeneratorFluctuating_878614: {MVStation_239: {'type': 'line',
   'line': Line_2390011}},
 GeneratorFluctuating_878615: {MVStation_239: {'type': 'line',
   

The dictionary you got should look something like that:
```python
{Generator_x: {BranchTee_y: {'type': 'line', 'line': Line_1}},
 BranchTee_y: {
  Generator_x: {'type': 'line', 'line': Line_1},
  BranchTee_z: {'type': 'line', 'line': Line_2}}
```

That means that Generator_x is connected to BranchTee_y by Line_1 and BranchTee_y is also connected to BranchTee_z by Line_2. Line_1 and Line_2 are [Line](http://edisgo.readthedocs.io/en/dev/api/edisgo.grid.html#edisgo.grid.components.Line) objects containig all important information about the line, such as length, equipment type, and geometry. Accessing this information can for example be done as follows:

In [7]:
edge_dictionary = edisgo.network.mv_grid.graph.edge
# get random line
line = edge_dictionary.popitem()[1].popitem()[1]['line']
# get line length
line.length

0.3681789122707058

Now let's have a look at all the **nodes**.

In [8]:
# get a list of all nodes (stations, generators, loads, branch tees)
# here, only the first 10 nodes are displayed
edisgo.network.mv_grid.graph.nodes()[:10]

[GeneratorFluctuating_839361,
 GeneratorFluctuating_839362,
 GeneratorFluctuating_839363,
 GeneratorFluctuating_839364,
 GeneratorFluctuating_878450,
 GeneratorFluctuating_878583,
 GeneratorFluctuating_878609,
 GeneratorFluctuating_878611,
 GeneratorFluctuating_878614,
 GeneratorFluctuating_878615]

You can also filter for certain kinds of nodes, e.g. generators...

In [9]:
# get a list of all generators in the mv grid
edisgo.network.mv_grid.graph.nodes_by_attribute('generator')

[GeneratorFluctuating_839361,
 GeneratorFluctuating_839362,
 GeneratorFluctuating_839363,
 GeneratorFluctuating_839364,
 GeneratorFluctuating_878450,
 GeneratorFluctuating_878583,
 GeneratorFluctuating_878609,
 GeneratorFluctuating_878611,
 GeneratorFluctuating_878614,
 GeneratorFluctuating_878615,
 GeneratorFluctuating_878862,
 GeneratorFluctuating_878863,
 GeneratorFluctuating_878864,
 GeneratorFluctuating_878865,
 GeneratorFluctuating_878866,
 GeneratorFluctuating_878867,
 GeneratorFluctuating_878875,
 GeneratorFluctuating_878950,
 GeneratorFluctuating_878963]

... or get a list of all lv grids.

In [10]:
# get a list of all lv grids
# here, only the first 10 lv grids are displayed
list(edisgo.network.mv_grid.lv_grids)[:10]

[LVGrid_122408,
 LVGrid_485974,
 LVGrid_138585,
 LVGrid_119895,
 LVGrid_119896,
 LVGrid_119889,
 LVGrid_119890,
 LVGrid_119891,
 LVGrid_119892,
 LVGrid_119893]

## Future generator capacities <a class="anchor" id="generator_scenario"></a>

In the open_eGo project we developed two future scenarios, the 'NEP 2035' and the 'ego 100' scenario. The 'NEP 2035' scenario closely follows the B2-Scenario 2035 from the German network developement plan (Netzentwicklungsplan NEP) 2015. The share of renewables is 65.8%, electricity demand is assumed to stay the same as in the status quo. The 'ego 100' scenario is based on the e-Highway 2050 scenario X-7 and assumes a share of renewables of 100% and again an equal electricity demand as in the status quo.

As mentioned earlier, ding0 grids represent status quo networks and generator capacities. In order to analyse future scenarios the future generator park has to be imported.

In [11]:
# Import generators
scenario = 'nep2035'
edisgo.import_generators(generator_scenario=scenario)

ProgrammingError: (psycopg2.ProgrammingError) relation "model_draft.ego_supply_res_powerplant_nep2035_mview" does not exist
LINE 2: FROM model_draft.ego_supply_res_powerplant_nep2035_mview 
             ^
 [SQL: 'SELECT model_draft.ego_supply_res_powerplant_nep2035_mview.id, model_draft.ego_supply_res_powerplant_nep2035_mview.subst_id, model_draft.ego_supply_res_powerplant_nep2035_mview.la_id, model_draft.ego_supply_res_powerplant_nep2035_mview.mvlv_subst_id, model_draft.ego_supply_res_powerplant_nep2035_mview.electrical_capacity, model_draft.ego_supply_res_powerplant_nep2035_mview.generation_type, model_draft.ego_supply_res_powerplant_nep2035_mview.generation_subtype, model_draft.ego_supply_res_powerplant_nep2035_mview.voltage_level, ST_AsText(ST_Transform(model_draft.ego_supply_res_powerplant_nep2035_mview.rea_geom_new, %(ST_Transform_1)s)) AS geom, ST_AsText(ST_Transform(model_draft.ego_supply_res_powerplant_nep2035_mview.geom, %(ST_Transform_2)s)) AS geom_em \nFROM model_draft.ego_

You can have a look at all generators again and compare it to the list of generators created earlier before the import of new generators.

In [ ]:
edisgo.network.mv_grid.graph.nodes_by_attribute('generator')

## Grid reinforcement <a class="anchor" id="grid_reinforcement"></a>

Now we can finally calculate grid expansion costs.

The grid expansion methodology is based on the distribution grid study of dena [[1]](#[1]) and Baden-Wuerttemberg [[2]](#[2]). For now only a combined analysis of MV and LV grids is possible. The order grid expansion measures are conducted is as follows:

* Reinforce transformers and lines due to over-loading issues
* Reinforce lines in MV grid due to over-voltage issues
* Reinforce lines in LV grid due to over-loading issues
* Reinforce transformers and lines due to over-loading issues

Reinforcement of transformers and lines due to over-loading issues is performed twice, once in the beginning and again after fixing over-voltage problems, because the changed power flows after reinforcing the grid may lead to new over-loading issues. (For further explanation see the [documentation](http://edisgo.readthedocs.io/en/dev/features_in_detail.html#automatic-grid-expansion).)

After each reinforcement step a non-linear power flow analyses is conducted using PyPSA. Let's do a power flow analysis before the reinforcement to see how many over-voltage issues there are.

In [12]:
# Do non-linear power flow analysis with PyPSA
edisgo.analyze()

INFO:pypsa.pf:Slack bus for sub-network 0 is Bus_MVStation_239
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots DatetimeIndex(['1970-01-01'], dtype='datetime64[ns]', freq='H')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000001 in 0.519885 seconds


In [13]:
# get voltage at each node from power-flow analysis results
v_mag_pu_pfa = edisgo.network.results.v_res(nodes=edisgo.network.mv_grid.graph.nodes())
# set maximum allowed voltage deviation to 10%
max_v_dev = 0.1
# find all nodes with a node voltage deviation greater the allowed voltage deviation
v_mag_pu_pfa[(v_mag_pu_pfa > (1 + max_v_dev))] - 1

Voltage levels for ['GeneratorFluctuating_839361', 'GeneratorFluctuating_839362', 'GeneratorFluctuating_839363', 'GeneratorFluctuating_839364', 'GeneratorFluctuating_878450', 'GeneratorFluctuating_878583', 'GeneratorFluctuating_878609', 'GeneratorFluctuating_878611', 'GeneratorFluctuating_878614', 'GeneratorFluctuating_878615', 'GeneratorFluctuating_878862', 'GeneratorFluctuating_878863', 'GeneratorFluctuating_878864', 'GeneratorFluctuating_878865', 'GeneratorFluctuating_878866', 'GeneratorFluctuating_878867', 'GeneratorFluctuating_878875', 'GeneratorFluctuating_878950', 'GeneratorFluctuating_878963', 'BranchTee_MVGrid_239_1', 'BranchTee_MVGrid_239_2', 'BranchTee_MVGrid_239_3', 'BranchTee_MVGrid_239_4', 'BranchTee_MVGrid_239_5', 'BranchTee_MVGrid_239_6', 'BranchTee_MVGrid_239_7', 'BranchTee_MVGrid_239_8', 'BranchTee_MVGrid_239_9', 'BranchTee_MVGrid_239_10', 'BranchTee_MVGrid_239_11', 'BranchTee_MVGrid_239_12', 'BranchTee_MVGrid_239_13', 'BranchTee_MVGrid_239_14', 'BranchTee_MVGrid_239_

1970-01-01


Reinforcement is invoked doing the following:

In [ ]:
# Do grid reinforcement
edisgo.reinforce()

Let's check for over-voltage issues again:

In [ ]:
# get voltage at each node from power-flow analysis results
v_mag_pu_pfa = edisgo.network.results.v_res(nodes=edisgo.network.mv_grid.graph.nodes())
# set maximum allowed voltage deviation to 10%
max_v_dev = 0.1
# find all nodes with a node voltage deviation greater the allowed voltage deviation
v_mag_pu_pfa[(v_mag_pu_pfa > (1 + max_v_dev))] - 1

## Evaluate results <a class="anchor" id="evaluation"></a>

Above we already saw how to access voltage results from the power flow analysis. All results are stored in the [Results](http://edisgo.readthedocs.io/en/dev/api/edisgo.grid.html#edisgo.grid.network.Results) object and can be accessed through
```python
edisgo.network.results
```

All changes in the grid conducted during the grid reinforcement, such as removed and new lines and new transformers, can be viewed as follows:

In [ ]:
edisgo.network.results.equipment_changes

You can also retrieve grid expansion costs through:

In [ ]:
costs = edisgo.network.results.grid_expansion_costs

If you are interested in accumulated costs you could group them like that:

In [ ]:
# group costs by type
costs_grouped = costs.groupby(['type']).sum()

An overview of the assumptions used to calculate grid expansion costs can be found in the [documentation]( http://edisgo.readthedocs.io/en/dev/features_in_detail.html#grid-expansion-costs).

Now let's compare the grid expansion costs for the 'NEP 2035' scenario with grid expansion costs for the 'ego 100' scenario. Therefore, we first have to setup the new scenario and calculate grid expansion costs.

In [ ]:
# initialize new EDisGo object with 'ego 100' scenario
edisgo_ego100 = EDisGo(ding0_grid=ding0_grid,
                       worst_case_analysis=worst_case_analysis,
                       generator_scenario='ego100')
# conduct grid reinforcement
edisgo_ego100.reinforce()
# get grouped costs
costs_grouped_ego100 = edisgo_ego100.network.results.grid_expansion_costs.groupby(['type']).sum()

In [ ]:
# plot

## References <a class="anchor" id="references"></a>

<a class="anchor" id="[1]"></a> [1] A.C. Agricola et al.: dena-Verteilnetzstudie: Ausbau- und Innovationsbedarf der Stromverteilnetze in Deutschland bis 2030. 2012.

<a class="anchor" id="[2]"></a> [2] C. Rehtanz et al.: Verteilnetzstudie für das Land Baden-Württemberg, ef.Ruhr GmbH, 2017.